In [3]:
%load_ext autoreload
%autoreload 2
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
import rpy2
from pprint import pprint
import re
import numpy as np
import pickle
from collections import OrderedDict
from itertools import repeat, count, product
from skbio.alignment import StripedSmithWaterman, local_pairwise_align_ssw
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
SRATE = 5000

In [6]:
robjects.r['load']("/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mtDNA_mappable_called_with_rawSignal.Rdata")

<StrVector - Python:0x7f4d2477aec8 / R:0x2f1b968>
['nanopore.bundled']

In [7]:
data = numpy2ri.ri2py(robjects.r['nanopore.bundled'])

In [8]:
files = !find ../../../../david_eccles_bc_ideas/processed/ | grep fast5

### Get data from RData

0 = reference
1 = called
3 = channel
8 = metrichor results (events)
9 = "mux"
10 = signal range 
11 = signal

In [9]:
ref = data[0][0]
channel_ids = [int(x) for x in data[3]]
file_ids = [int(x) for x in data[5]]
types = list(data[2].names) #template/2D/...

In [31]:
## channel coords from rdata
coords = []
all_raw = {}
channel_max = dict(zip(channel_ids, repeat(0)))
for i in range(0, len(data[10]), 2):
    se = (round(data[10][i]), round(data[10][i+1]))
    coords.append(se)
    channel_max[channel_ids[int(i/2)]] = max(channel_max[channel_ids[int(i/2)]], se[1])
    
print (channel_max)

for chid, chmax in channel_max.items():
    all_raw[chid] = np.empty(chmax)
    all_raw[chid].fill(np.nan)

{195: 4638530, 132: 69987497, 133: 12456817, 135: 62976700, 201: 4599826, 227: 18246574, 203: 6371298, 204: 13066295, 141: 29953471, 142: 53380406, 206: 55940501, 209: 25813234, 211: 62405297, 212: 35840257, 149: 57367518, 215: 59538461, 223: 67207073, 156: 63222798, 157: 7725489, 213: 35016383}


pack all the available raw signal for the given channel in one array. 

In [32]:
for i, start_end in enumerate(coords):
    start, end = start_end
    print(i, start)
    for k, sig in enumerate(data[11][i]):
        all_raw[channel_ids[i]][start + k] = sig

0 14263383
1 14263383
2 53054262
3 55559668
4 55763736
5 55559668
6 20841985
7 21744018
8 20841663
9 58611017
10 29733238
11 30178940
12 38452561
13 68761506
14 69336966
15 39203872
16 4338899
17 10117943
18 10111547
19 1610262
20 1610262
21 437216
22 1290055
23 25820667
24 28685564
25 29586312
26 28684851
27 36094944
28 36495057
29 36094944
30 38381291
31 60958967
32 60827554
33 62183810
34 13895041
35 14282593
36 14883605
37 16839366
38 17196412
39 11116981
40 12018853
41 28113526
42 50760609
43 52030192
44 11786742
45 11971416
46 4415895
47 14826058
48 15605747
49 17611679
50 15605747
51 50100827
52 50279488
53 50100827
54 54684243
55 56708683
56 62911375
57 4682908
58 9660018
59 10549882
60 9660018
61 1262851
62 2095322
63 3367671
64 5777375
65 3310668
66 4523542
67 3724151
68 2837075
69 12054328
70 2865115
71 4214741
72 5115935
73 12445897
74 13721867
75 16388228
76 15418914
77 6253118
78 6213608
79 8839630
80 9700201
81 8839630
82 27325701
83 28050421
84 27325701
85 30627084
86 5

In [33]:
pickle.dump(all_raw, open("/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_raw.pickle", 'wb'), protocol=2)

In [30]:
[(i, x) for i, x in enumerate(all_raw[channel_ids[0]]) if x < -1]

[(5077275, -12.0),
 (14527754, -38.0),
 (14537810, -20.0),
 (14988691, -46.0),
 (21204879, -4096.0),
 (21204880, -889.0),
 (21204881, -464.0),
 (21204882, -234.0),
 (21204883, -107.0),
 (21204884, -34.0),
 (21904280, -12.0),
 (22352821, -45.0),
 (29865513, -43.0),
 (30162890, -38.0),
 (30463523, -25.0),
 (30613377, -79.0),
 (30764920, -10.0),
 (38883202, -23.0),
 (39782445, -80.0),
 (40236039, -8.0),
 (40267614, -1647.0),
 (40267615, -1164.0),
 (40267616, -563.0),
 (40267617, -265.0),
 (40267618, -112.0),
 (40267619, -13.0),
 (40835307, -2.0),
 (41133529, -24.0),
 (41282610, -56.0),
 (41438277, -60.0),
 (53149625, -117.0),
 (53454479, -4096.0),
 (53752870, -122.0),
 (54054085, -137.0),
 (54202847, -25.0),
 (54353318, -91.0),
 (54664921, -10.0),
 (54965536, -126.0),
 (55113689, -90.0),
 (55263463, -97.0),
 (55563561, -117.0),
 (55712566, -90.0),
 (58718623, -42.0),
 (59168470, -35.0),
 (59321017, -40.0),
 (59621103, -10.0),
 (59732717, -4096.0),
 (59732718, -815.0),
 (59732719, -389.0),